[View in Colaboratory](https://colab.research.google.com/github/lukasheinrich/pyhf-benchmarks/blob/master/colab/CPU_standalone.ipynb)

# CPU pyhf interpolation

In [2]:
import numpy as np
import timeit
import tensorflow as tf
def setup(N,float_t):
    def _hfinterp_code1(histogramssets, alphasets):
        allset_allhisto_deltas_up = tf.divide(histogramssets[:,:,2], histogramssets[:,:,1])
        allset_allhisto_deltas_dn = tf.divide(histogramssets[:,:,0], histogramssets[:,:,1])

        def ones(shape):
            return tf.ones(shape, dtype = float_t)
        def zeros(shape):
            return tf.zeros(shape, dtype = float_t)
        def where(mask,a,b):
            mask = tf.cast(mask, float_t)
            inv_mask = tf.cast(1-mask, float_t)
            return mask * a + inv_mask * b
        
        allsets_allhistos_masks = where(alphasets > 0, ones(alphasets.shape), zeros(alphasets.shape))
        bases_up = tf.einsum('sa,shb->shab', ones(alphasets.shape), allset_allhisto_deltas_up)
        bases_dn = tf.einsum('sa,shb->shab', ones(alphasets.shape), allset_allhisto_deltas_dn)
        exponents = tf.einsum('sa,shb->shab', tf.abs(alphasets), ones(allset_allhisto_deltas_up.shape))
        masks = tf.einsum('sa,shb->shab', allsets_allhistos_masks, ones(allset_allhisto_deltas_up.shape))

        bases = where(masks, bases_up, bases_dn)
        return tf.pow(bases, exponents)

    
    a_shape = (100,1)
    h_shape = (100,100,3,N)

    with tf.device('/cpu:0'):
      a = tf.placeholder(float_t, a_shape)
      h = tf.placeholder(float_t, h_shape)
      gpu_op = _hfinterp_code1(h,a)

    return [gpu_op], [a,h], [a_shape,h_shape]

def run_it(ops,args,shapes):
    r = session.run(ops, {args[0]: np.random.uniform(-1,1, size = shapes[0]), args[1]: np.random.uniform(-1,1, size = shapes[1])})
    return r

results = []
for n in np.linspace(500,7000,14):
    n = int(n)
    session = tf.Session()
    try:
        print('Running ops')
        exec_time = timeit.timeit('run_it(o,a,s)', number=10, setup="from __main__ import run_it, setup; import tensorflow as tf; o,a,s = setup({},tf.float16)".format(n))
        results.append((n,exec_time))
        print('N: {} time: {}'.format(n,exec_time))
    finally:
        # For now, TPU sessions must be shutdown separately from
        # closing the session.
        session.close()


Running ops
N: 500 time: 24.69131860000016
Running ops
N: 1000 time: 50.953526545001296
Running ops
N: 1500 time: 76.26330991199939
Running ops
N: 2000 time: 101.76752162900084
Running ops
N: 2500 time: 126.98815078100051
Running ops
N: 3000 time: 152.69746947000021
Running ops
N: 3500 time: 178.52999462200023
Running ops
N: 4000 time: 203.67025582600036
Running ops
N: 4500 time: 230.400485225
Running ops
N: 5000 time: 256.978021799001
Running ops
N: 5500 time: 282.4864051000004
Running ops
N: 6000 time: 308.44447330899857
Running ops
N: 6500 time: 334.29839917199934
Running ops
N: 7000 time: 360.149083409


In [3]:
results

[(500, 24.69131860000016),
 (1000, 50.953526545001296),
 (1500, 76.26330991199939),
 (2000, 101.76752162900084),
 (2500, 126.98815078100051),
 (3000, 152.69746947000021),
 (3500, 178.52999462200023),
 (4000, 203.67025582600036),
 (4500, 230.400485225),
 (5000, 256.978021799001),
 (5500, 282.4864051000004),
 (6000, 308.44447330899857),
 (6500, 334.29839917199934),
 (7000, 360.149083409)]